# Generacion Features de Grafo

## Imports & Load Data

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import geopandas as gpd
import networkx as nx

import numpy as np
import pandas as pd

import pickle

In [4]:
# Cargar endpoint
infile = open('./data/1_GDB_Layers.pkl','rb')
GDB = pickle.load(infile)
infile.close()

In [5]:
filename = './data/22_Full_Graph.pkl'
infile = open(filename, 'rb')
MTG = pickle.load(infile)
infile.close()

In [6]:
filename = './data/22_CTMT_Graph_Dict.pkl'
infile = open(filename, 'rb')
CTMT_GR = pickle.load(infile)
infile.close()

In [7]:
filename = './data/22_UNTRD_Conections.pkl'
infile = open(filename, 'rb')
UNTRD_CONS = pickle.load(infile)
infile.close()

## Obtener Rutas a Sub Estacion

In [8]:
def get_route(trd):
    route = []
    
    if trd.UNTRD_PCON == '':
        return route
    
    try:
        route = nx.shortest_path(CTMT_GR[trd.CTMT], trd.UNTRD_PCON, trd.CTMT_PCON, weight='length')
    except Exception as e:
        print('-----------')
        print(e)
        print('Probando con grafo Gral')
        try:
            route = nx.shortest_path(MTG, trd.UNTRD_PCON, trd.CTMT_PCON, weight='length')
        except Exception as e:
            print(e)
            print(f'CTMT:{trd.CTMT}, Source:{trd.UNTRD_PCON}, Target:{trd.CTMT_PCON}')
    
    return route

In [9]:
UNTRD_CONS['ROUTE_SUB'] = UNTRD_CONS.apply(get_route, axis=1)

-----------
No path to nan.
Probando con grafo Gral
No path to nan.
CTMT:27117492, Source:27128440, Target:nan
-----------
No path to 27863987.
Probando con grafo Gral
No path to 27863987.
CTMT:27117487, Source:27148623, Target:27863987
-----------
No path to 27707915.
Probando con grafo Gral
No path to 27707915.
CTMT:27117496, Source:27149709, Target:27707915


In [10]:
## Ruta hasta sub estacion referenciado a los ID de los segmentos de SSDMT
UNTRD_CONS['SID_ROUTE_SUB'] = UNTRD_CONS.apply(
    lambda x: [at['COD_ID'] for p1,p2,at in CTMT_GR[x.CTMT].edges(x.ROUTE_SUB, data=True)], axis=1)

In [11]:
UNTRD_CONS.shape

(3848, 8)

In [12]:
# Pre filtrar variables de interes
SSDMT_ft = ['COD_ID','TIP_CND','COMP']
SEGCON_ft = ['COD_ID', 'R1', 'X1', 'FTRCNV', 'CNOM', 'CMAX'] 

In [13]:
# Cargar valores electricos relacionados a los segmentos de transmicion
SSDMT_ELE = pd.merge(GDB['SSDMT'][SSDMT_ft], GDB['SEGCON'][SEGCON_ft],
                     how='left', left_on='TIP_CND', right_on='COD_ID', suffixes=('','CON'))

## Adicionar Informacion Electrica

In [14]:
def get_elec_features(TRD):
    
    SEG_REF_ROUTE = TRD.SID_ROUTE_SUB
    
    SRR_ELE = SSDMT_ELE[SSDMT_ELE.COD_ID.isin(SEG_REF_ROUTE)]
    
    DISTANCIA = SRR_ELE.COMP.sum()
    RESISTENCIA = (SRR_ELE.R1  * SRR_ELE.COMP).sum()
    REACTANCIA = (SRR_ELE.X1 * SRR_ELE.COMP).sum()
    
    FCONV = SRR_ELE.FTRCNV.mean()
    
    I_NOM_MIN = SRR_ELE.CNOM.min()
    I_NOM_MED = SRR_ELE.CNOM.median()
    
    I_MAX_MIN = SRR_ELE.CMAX.min()
    I_MAX_MED = SRR_ELE.CMAX.median()
    
    elect_ft = {
        'DISTANCE_SUB': DISTANCIA,
        'RESI_SUB': RESISTENCIA,
        'REAC_SUB': REACTANCIA,
        'FCONV_R': FCONV,
        'I_NOM_MED_R': I_NOM_MED,
        'I_NOM_MIN_R': I_NOM_MIN,
        'I_MAX_MED_R': I_MAX_MED,
        'I_MAX_MIN_R': I_MAX_MIN
    }
    
    return elect_ft

In [15]:
UNTRD_ELE = UNTRD_CONS.apply(get_elec_features , axis=1, result_type='expand')

In [16]:
# Agregar Variables Manualmente
UNTRD_ELE['Z_MOD_SUB'] = UNTRD_ELE.apply(lambda x: np.absolute(x.RESI_SUB - x.REAC_SUB*1j), axis=1)
UNTRD_ELE['Z_ANG_SUB'] = UNTRD_ELE.apply(lambda x: np.angle(x.RESI_SUB- x.REAC_SUB*1j, deg=True), axis=1)

## Unir variables de distancia y electricas

In [17]:
UNTRD_CONS[UNTRD_ELE.columns] = UNTRD_ELE

In [18]:
UNTRD_CONS.sample(5)

,COD_ID,PAC_1,CTMT,SUB,UNTRD_PCON,CTMT_PCON,ROUTE_SUB,SID_ROUTE_SUB,DISTANCE_SUB,FCONV_R,I_MAX_MED_R,I_MAX_MIN_R,I_NOM_MED_R,I_NOM_MIN_R,REAC_SUB,RESI_SUB,Z_MOD_SUB,Z_ANG_SUB
2556,26877238,75471000754036300ET,27117485,27335509,27135985,27152908,"[27135985, 27136025, 27135986, 27136026, 27136...","[26693150, 26693135, 26699498, 26680819, 26693...",2112.691,0.382188,230.0,97.0,230.0,97.0,834.648623,1116.376534,1393.891993,-36.783289
597,26878681,75783800752819100ET,27117496,27335510,27142846,27707915,"[27142846, 27158704, 27152184, 27151942, 27151...","[26704644, 26683074, 26683640, 26685442, 26683...",10664.952,0.448240,365.0,97.0,365.0,97.0,4096.062174,4119.925692,5809.605240,-44.833584
1911,26878468,75927300753568600ET,27117495,27335510,27144841,27131280,"[27144841, 27158814, 27158433, 27158177, 27160...","[26695649, 26703202, 26684375, 26708156, 26687...",5668.202,0.576429,438.0,95.0,438.0,95.0,2220.989902,2603.227207,3421.927532,-40.469746
1540,26877738,75242200753668700ET,80345444,27335511,27140446,27708175,"[27140446, 27143649, 27143313, 27140760, 27143...","[26706015, 26690881, 26684995, 26690812, 26706...",5451.810,0.457000,438.0,54.0,438.0,54.0,2055.654511,1685.511820,2658.320064,-50.650329
3006,26876848,76818222752570311MT,27117484,27335513,27158993,44571767,"[27158993, 27159479, 27158992, 27158991, 27161...","[26700505, 26707041, 26700504, 26700503, 26700...",19293.869,0.459458,438.0,54.0,438.0,54.0,7758.728863,16790.072494,18496.064660,-24.801761


In [19]:
UNTRD_CONS.shape

(3848, 18)

In [20]:
UNTRD_CONS.columns

Index(['COD_ID', 'PAC_1', 'CTMT', 'SUB', 'UNTRD_PCON', 'CTMT_PCON',
       'ROUTE_SUB', 'SID_ROUTE_SUB', 'DISTANCE_SUB', 'FCONV_R', 'I_MAX_MED_R',
       'I_MAX_MIN_R', 'I_NOM_MED_R', 'I_NOM_MIN_R', 'REAC_SUB', 'RESI_SUB',
       'Z_MOD_SUB', 'Z_ANG_SUB'],
      dtype='object')

## Exportar exit point

In [20]:
filename = './data/222_ETRD_ROUTES_ELEC.pkl'
outfile = open(filename, 'wb')
pickle.dump(UNTRD_CONS,outfile)
outfile.close()